In [54]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np

In [55]:
DownloadsPath = 'C:/Users/User/Downloads/'
FileName = 'Задание (данные)'
FileType = 'xlsx'
File_path = f"{DownloadsPath}{FileName}.{FileType}"

In [56]:
data = pd.read_excel(File_path)

In [59]:
NoPresentation = data[['No Presentation','Unnamed: 1']]
Presentation = data[['Presentation','Unnamed: 4']]


NoPresentation.columns = NoPresentation.iloc[0]
NoPresentation = NoPresentation[1:]

Presentation.columns = Presentation.iloc[0]
Presentation = Presentation[1:]
Presentation  = Presentation.dropna()

Presentation.head()

,Client ID,Profit
1,43,633.9
2,257,406.61
3,477,382.2
4,754,331.25
5,825,456.01


### Убираем выбросы

In [60]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        #print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
    return df

In [63]:
PresentationOutliers = remove_outliers(Presentation,['Profit'],3)
NoPresentationOutliers = remove_outliers(NoPresentation,['Profit'],3)

PM = PresentationOutliers.Profit.mean()
PN = PresentationOutliers.shape[0]
PD =PresentationOutliers.Profit.std()

NPM = NoPresentationOutliers.Profit.mean()
NPD = NoPresentationOutliers.Profit.std()
NPN = PresentationOutliers.shape[0]

print(NPM, PM)

583.112037735849 547.1930567685589


Убранные выбросы не повлияли на среднее значение. А значит и наш вывод не меняется - ищем другие причины 

## Нужно проверить 2 гипотезы: 
1. Нет стат.значимого эффекта и значит презентация не имела не положительно, не отрицатлеьного эффекта
2. Эффект есть, но изначальная выборка была не репрезентативна

In [22]:
Z = (NPM-PM) / (np.sqrt(NPM**2/NPN) + np.sqrt(PM**2/PN))

In [23]:
Z

0.48089035112626205

Так как Z < 1,64 то можно с уверенностью заявить, что статзначимого эффекта нет и по крайней мере наше общение с клиентами не повредило нашим продажам
Проверяли на 95% уровне значимости

In [64]:
data = pd.read_excel("C:/Users/User/Downloads/Задание (данные).xlsx",'Table 2',header=1)
data.head()

,Client ID,Age,Gender,Monthly Income
0,43,37,M,3905.48
1,137,37,M,4665.64
2,257,39,M,4744.88
3,477,36,M,3815.35
4,754,31,M,3916.61


In [29]:
data.describe()

,Client ID,Age,Monthly Income
count,500.000000,500.000000,500.000000
mean,48583.270000,37.836000,5178.765960
std,29867.474223,4.170358,1645.226433
min,43.000000,30.000000,2062.900000
25%,22015.750000,34.000000,3991.555000
50%,48641.000000,38.000000,4935.305000
75%,73771.750000,42.000000,6069.720000
max,99744.000000,45.000000,13230.150000


Как видно возрастная группа в целом однородная. Можно предположить, что 30 и 45 летние отличаются не значительно.
Хотя можно предположить, что более взрослые получают больший доход в месяц, так что стоило разбивать именно по нему
Так же можно предположить, что есть какая-то модель, которая задает доход в зависимости от возраста, вероятно даже не линейно
Так же вероятно, что женщины получают меньший доход в месяц, так как у нас нет других вводных кроме возраста и пола, 
как сфера деятельности, наличие высшего образования, колличество лет опыта

# Померджим таблицы друг на друга

Что бы можно было сравнить отличаются ли наши выборки

In [65]:
PresentationM = data.merge(PresentationOutliers,how='inner', on = 'Client ID')
NoPresentationM =  data.merge(NoPresentationOutliers,how='inner', on = 'Client ID')

In [66]:
PresentationM.describe()

,Age,Monthly Income
count,229.000000,229.000000
mean,37.628821,4534.397074
std,4.160935,1269.195780
min,30.000000,2062.900000
25%,34.000000,3666.380000
50%,38.000000,4377.560000
75%,41.000000,5255.260000
max,45.000000,8974.530000


In [67]:
NoPresentationM.describe()

,Age,Monthly Income
count,269.000000,269.000000
mean,37.996283,5694.619703
std,4.137107,1688.939158
min,30.000000,2181.830000
25%,34.000000,4538.930000
50%,38.000000,5453.910000
75%,42.000000,6684.940000
max,45.000000,11377.910000


In [68]:
Z = (NoPresentationM.mean()-PresentationM.mean()) / (np.sqrt(NoPresentationM.std()**2/NoPresentationM.count()) + np.sqrt(PresentationM.std()**2/PresentationM.count()))

In [69]:
Z['Monthly Income']

6.20947106499005

Так как Z значение для месячного дохода больше 1.64  - критического значения на уровня значимости 5%, то видно, что выборки не идентичны их их использование для проверки нашей гипотезы не подходят

Что бы все же не проводить новый эксперимент и не тратить время сотрудников и деньги компании можем посмотреть какую долю от своего месчного дохода тратя люди в каждой из групп

In [72]:
PresentationM['Percentage'] =  PresentationM['Profit'] / PresentationM['Monthly Income']
PresentationM.head()

,Client ID,Age,Gender,Monthly Income,Profit,Percentage
0,43,37,M,3905.48,633.9,0.16231
1,257,39,M,4744.88,406.61,0.085694
2,477,36,M,3815.35,382.2,0.100174
3,754,31,M,3916.61,331.25,0.084576
4,825,31,M,3211.30,456.01,0.142002


In [73]:
NoPresentationM['Percentage'] =  NoPresentationM['Profit'] / NoPresentationM['Monthly Income']
NoPresentationM.head()

,Client ID,Age,Gender,Monthly Income,Profit,Percentage
0,137,37,M,4665.64,380.37,0.081526
1,1809,40,M,4816.03,333.11,0.069167
2,1929,38,F,2980.46,438.91,0.147263
3,1976,32,M,4691.48,383.21,0.081682
4,2057,33,M,4305.03,147.46,0.034253


In [74]:
Z = (NoPresentationM.mean()-PresentationM.mean()) / (np.sqrt(NoPresentationM.std()**2/NoPresentationM.count()) + np.sqrt(PresentationM.std()**2/PresentationM.count()))

In [75]:
abs(Z.Percentage)  

4.050298985546802

Берем модуль, так как из среднего по выборке где мы нечитаем лекцию вычитаем среднее по выборке где читаем
А мы планируем проверить, что наше действие возымело эффект

так как  наше значение больше 1,64 что можно сделать вывод, что наха компания прошла успешно, так как на уровне значимости 5% гипотеза о равенстве средней доли потраченной в нашем продукте отвергается, и даже на уровне значимости 1%, так что точно можно быть уверенны